# Day 0: Imports and Utility Functions

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import math
import numpy as np
import os
import operator
import re
import random
import string
import collections, itertools, functools, heapq, operator, bisect
from collections import Counter, defaultdict, namedtuple, deque, OrderedDict
from functools   import lru_cache, reduce
from statistics  import mean, median, mode, stdev, variance
from itertools   import (permutations, combinations, groupby, repeat,
                         islice, repeat,cycle, chain, product, zip_longest, takewhile, dropwhile, count as count_from)
from heapq       import heappush, heappop, heapify
from operator    import iand, ior, ilshift, irshift
from bisect      import bisect_left, bisect_right

In [2]:
input_path = lambda day : "inputs/day{}.txt".format(day)
input_path_sample = lambda day : "inputs/day{}-sample.txt".format(day)

In [3]:
def get_ints(f):
    """extract ints from a file."""
    return map(int, re.findall(r'[+-]?[0-9]+', f.read()))

# [Day 1: Chronal Calibration](https://adventofcode.com/2018/day/1)

In [4]:
with open(input_path(1), 'r') as f:
    changes = list(get_ints(f))

In [5]:
print(sum(changes))
seen = {0}
freq = 0
for change in itertools.cycle(changes):
    freq += change
    if freq in seen:
        print(freq)
        break
    seen.add(freq)

582
488


# [Day 2: Inventory Management System](https://adventofcode.com/2018/day/2)

In [6]:
with open(input_path(2), 'r') as f:
    boxes = [line.strip() for line in f]

In [7]:
def compute_checksum(boxes):
    """compute checksum of boxes."""
    def check(box):
        counter = collections.Counter(box)
        return 2 in counter.values(), 3 in counter.values()
    two_cnt = three_cnt = 0
    for box in boxes:
        two, three = check(box)
        two_cnt += two
        three_cnt += three
    return two_cnt * three_cnt

In [8]:
print("Checksum is {}.".format(compute_checksum(boxes)))

Checksum is 6474.


In [9]:
nboxes = len(boxes)
for i, j in itertools.combinations(range(nboxes), 2):
    if sum(c1 != c2 for c1, c2 in zip(boxes[i], boxes[j])) == 1:
        print(''.join(c1 for c1, c2 in zip(boxes[i], boxes[j]) if c1 == c2))
        break

mxhwoglxgeauywfkztndcvjqr


# [Day 3: No Matter How You Slice It](https://adventofcode.com/2018/day/3)

In [10]:
with open(input_path(3), 'r') as f:
    lines = f.readlines()

In [11]:
Square = collections.namedtuple('Square', 'c0, r0, width, height')
def get_squares(lines):
    """get squares from lines."""
    squares = []
    for line in lines:
        c0, r0 = map(int, re.search(r'[\d]+,[\d]+', line).group().split(','))
        width, height = map(int, re.search(r'[\d]+x[\d]+', line).group().split('x'))
        squares.append(Square(c0, r0, width, height))
    return squares

In [12]:
squares = get_squares(lines)

In [13]:
def count_overlap_cells(squares):
    """count overlap cells of squares."""
    cells = collections.Counter()
    for c0, r0, width, height in squares:
        for r in range(r0, r0 + height):
            for c in range(c0, c0 + width):
                cells[r, c] += 1
    return sum(cnt != 1 for cnt in cells.values())

In [14]:
print(count_overlap_cells(squares))

115242


In [15]:
def find_nonoverlap_claim(squares):
    """find the claim which has no overlap square with others."""
    cells = collections.Counter()
    for c0, r0, width, height in squares:
        for r in range(r0, r0 + height):
            for c in range(c0, c0 + width):
                cells[r, c] += 1
    for claim_id, (c0, r0, width, height) in enumerate(squares, 1):
        if all(cells[r, c] == 1 for r in range(r0, r0 + height) for c in range(c0, c0 + width)):
            return claim_id

In [16]:
find_nonoverlap_claim(squares)

1046

# [Day 4: Repose Record](https://adventofcode.com/2018/day/4)

In [17]:
with open(input_path(4), 'r') as f:
    records = [line.strip() for line in f]

In [18]:
def analyze_guard_records(records):
    """analyze guard records and return sleep times for each guard."""
    lo = 0
    sleeps = collections.defaultdict(collections.Counter)
    for record in sorted(records):
        time = record[12:17]
        h, m = map(int, time.split(':'))
        content = record[18:].strip()
        if content.startswith('Guard'):
            _, guard_id, *_ = content.split()
            guard_id = int(guard_id[1:])
        elif content == 'falls asleep':
            lo = m
        else:
            assert(content == 'wakes up')
            for t in range(lo, m):
                sleeps[guard_id][t] += 1
    return sleeps

In [19]:
sleeps = analyze_guard_records(records)

## Strategy 1

In [20]:
best_guard_id = max(sleeps, key=lambda x : sum(sleeps[x].values()))

In [21]:
best_t = max(range(60), key=lambda t : sleeps[best_guard_id][t])

In [22]:
print(best_guard_id * best_t)

3212


## Strategy 2

In [23]:
best_guard_id = max(sleeps, key=lambda x : max(sleeps[x].values()))

In [24]:
best_t = max(range(60), key=lambda t : sleeps[best_guard_id][t])

In [25]:
print(best_guard_id * best_t)

4966


# [Day 5: Alchemical Reduction](https://adventofcode.com/2018/day/5)

In [26]:
with open(input_path(5), 'r') as f:
    polymer = f.read().strip()

In [27]:
print(len(polymer))

50000


In [28]:
def reduce_polymer(polymer, is_opposite_polarity_func):
    """return final polymer after reduction according to is_opposite_polarity function."""
    stack = []
    for ch in polymer:
        if stack and is_opposite_polarity_func(ch, stack[-1]):
            stack.pop()
        else:
            stack.append(ch)
    return ''.join(stack)

In [29]:
def is_opposite_polarity(c1, c2):
    """check if c1 and c2 has opposite polarity."""
    return c1 != c2 and c1.lower() == c2.lower()

In [30]:
reduced_polymer = reduce_polymer(polymer, is_opposite_polarity)

In [31]:
print(len(reduced_polymer))

11298


In [32]:
def shortest_polymer(polymer, is_opposite_polarity_func):
    """find shortest polymer by removing a character type."""
    mn = len(polymer) + 1
    for ch in string.ascii_lowercase:
        modified_polymer = polymer.replace(ch, '')
        modified_polymer = modified_polymer.replace(ch.upper(), '')
        mn = min(mn, len(reduce_polymer(modified_polymer, is_opposite_polarity_func)))
    return mn

In [33]:
print(shortest_polymer(polymer, is_opposite_polarity))

5148


# [Day 6: Chronal Coordinates](https://adventofcode.com/2018/day/6)